# 1. Generate TEST and TRAINING dataframes

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn import tree, cross_validation, ensemble
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import NearestCentroid
from sklearn.naive_bayes import GaussianNB, BernoulliNB

# Import TRAINING dataframe
train_df = pd.read_csv("../dataset/training_df_with_standardization.csv", sep="\t", header=0)

feature_labels = ['height', 'hips', 'waist', 'dress', 'shoes', 'insta_num_post', 'insta_avg_likes', 'insta_avg_comments', 'agencies']
train_indep = train_df[feature_labels]
train_dep = []
fshows = list(train_df.fashionshows)
for i in fshows:
    if i < 1: train_dep.append(0)
    else: train_dep.append(1)
        
print train_indep.head()

     height      hips     waist     dress     shoes  insta_num_post  \
0 -1.017854 -1.711893 -0.908382 -0.158134  0.483026       -0.523898   
1 -0.208052  0.593104 -0.908382 -0.158134  2.128892        0.573235   
2  0.601750 -0.559394  0.357150  0.658352 -0.751373       -0.503197   
3  0.196849 -1.250893 -0.149063  0.658352  0.483026       -0.027083   
4 -1.422755 -0.789894  0.357150 -0.158134 -0.339907        0.076420   

   insta_avg_comments  insta_avg_likes  agencies  
0           -0.025903        -0.119297         1  
1           -0.175820        -0.161911         1  
2           -0.154404        -0.155551         1  
3           -0.132987        -0.146329         1  
4            0.316764        -0.046471         1  


In [2]:
#Import TEST dataframe
test_df = pd.read_csv("../dataset/test_df_with_standardization.csv", sep="\t", header=0)
test_indep = test_df[['height', 'hips', 'waist', 'dress', 'shoes', 'insta_post', 'insta_likes', 'insta_comments', 'agencies']]
test_dep = list(test_df.fashionshows)
print test_indep

      height      hips     waist     dress     shoes  insta_post  insta_likes  \
0   0.196849 -0.559394 -1.161488 -0.158134  1.305959    0.242025    -0.126293   
1   0.196849 -0.789894 -0.149063 -0.158134  0.483026   -0.482496    -0.163184   
2   0.601750  0.132105  0.357150  0.658352 -0.339907   -0.979311    -0.173360   
3   1.006651  0.593104 -0.149063  0.658352  1.305959    0.097121    -0.143784   
4  -1.017854 -0.789894 -1.161488 -0.158134 -1.162840    2.394890    -0.151099   
5   1.006651  0.593104 -0.149063  0.658352  1.305959   -0.937910    -0.158095   
6   1.006651  1.054103  0.357150  0.658352  0.483026   -0.772305    -0.136470   
7   0.196849 -0.328895  0.357150 -0.158134 -0.339907   -0.275490    -0.144738   
8   2.221355 -0.789894  0.863363  0.658352  1.305959   -0.648101    -0.153961   
9  -0.612953 -0.328895 -0.655275 -0.158134 -0.751373   -0.627401     0.032398   
10 -0.208052 -0.789894 -0.149063 -0.974620 -0.751373   -0.192688    -0.157141   
11  1.006651  1.054103 -0.14

# 2. Testing with three classifiers

In [4]:
mlClassifier = [tree.DecisionTreeClassifier(criterion='entropy', max_depth=5), 
                ensemble.RandomForestClassifier(n_estimators=250, criterion='entropy', max_depth=5, max_features=5), 
                ensemble.AdaBoostClassifier(n_estimators=25)]
#mlClassifier = [tree.DecisionTreeClassifier(max_depth=5), ensemble.RandomForestClassifier(max_depth=5), ensemble.AdaBoostClassifier(n_estimators=25)]
dt = mlClassifier[0]
erf = mlClassifier[1]
eab = ensemble.AdaBoostClassifier(mlClassifier[0], n_estimators=100)
# adaboost = ensemble.AdaBoostClassifier(base_estimator= mlClassifier[1])
# gradient_descent = LogisticRegression(C=.01) # [0.95199999999999996, 0.94199999999999995, 0.96399999999999997, 0.95799999999999996, 0.90800000000000003, 0.94399999999999995, 0.95799999999999996, 0.96199999999999997, 0.95999999999999996, 0.93000000000000005, 0.95399999999999996, 0.95999999999999996, 0.93400000000000005, 0.92800000000000005, 0.93200000000000005] 0.848214285714

# base_clf = BernoulliNB(alpha=.001, binarize=0.0, class_prior=None, fit_prior=False) 
# base_clf = mlClassifier[0]
# eab = ensemble.AdaBoostClassifier(base_estimator=erf)
#indep_df = mlModel[md_count]

x_train = train_indep.loc[:]
y_train = train_dep

x_test = test_indep.loc[:]
y_test = test_dep

niter = 10

dt_result = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
erf_result = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
eab_result = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
eboo_result = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

feature_selection = [.0, .0, .0, .0, .0, .0, .0, .0, .0]

for i in range(niter):
    
    dt.fit(x_train, y_train)
    erf.fit(x_train, y_train)
    eab.fit(x_train, y_train)
#     boost.fit(x_train, y_train)

    importances = erf. feature_importances_ 
    std = np.std([tre.feature_importances_ for tre in erf.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]
#     print("Feature ranking:")
    
    for f in range(9):
#         print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
        feature_selection[indices[f]] += importances[indices[f]]
    
    dt_pred = dt.predict(x_test)
    erf_pred = erf.predict(x_test)
    eab_pred = eab.predict(x_test)
#     eboo_pred = boost.predict(x_test)
        
    dt_result = [float(x + y) for x, y in zip(dt_result, dt_pred)]
    erf_result = [float(x + y) for x, y in zip(erf_result, erf_pred)]
    eab_result = [float(x + y) for x, y in zip(eab_result, eab_pred)]
#     eboo_result = [x + y for x, y in zip(eboo_result, eboo_pred)]

dt_result = [x / niter for x in dt_result]
erf_result = [x / niter for x in erf_result]
eab_result = [x / niter for x in eab_result]
# eboo_result = [(x*1.0) / niter for x in eboo_result]
fs_result = [x / niter for x in feature_selection]

print y_test
print dt_result, roc_auc_score(y_test, dt_result)
print erf_result, roc_auc_score(y_test, erf_result)
print eab_result, roc_auc_score(y_test, eab_result)
# print eboo_result, roc_auc_score(y_test, eboo_result) # eboo_result[:, 1]

print fs_result
print("Feature importance:")
for f in range(9):
    print("%s (%f)" % (feature_labels[f], fs_result[f]))

[1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0.5625
[0.6, 0.0, 0.0, 0.0, 0.0, 0.2, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0.6875
[0.7, 0.7, 0.0, 0.0, 0.0, 0.9, 0.0, 1.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.9] 0.598214285714
[0.2106411947799621, 0.057928611076750104, 0.079440543585006379, 0.033427467440480675, 0.050636780605703161, 0.12846905037259787, 0.12856185102704712, 0.16534801722564554, 0.14554648388680708]
Feature importance:
height (0.210641)
hips (0.057929)
waist (0.079441)
dress (0.033427)
shoes (0.050637)
insta_num_post (0.128469)
insta_avg_comments (0.128562)
insta_avg_likes (0.165348)
agencies (0.145546)


In [5]:
print classification_report(y_test, eab_result)

ValueError: Mix type of y not allowed, got types set(['binary', 'continuous'])

In [87]:
int_eboo =[round(x) for x in eboo_result]
print int_eboo
print classification_report(y_test, int_eboo)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
             precision    recall  f1-score   support

          0       0.47      1.00      0.64         7
          1       0.00      0.00      0.00         8

avg / total       0.22      0.47      0.30        15



# 3. Output the result in Latex format

In [ ]:
test_indep['runway'] = y_test
test_indep['decision tree'] = dt_result
test_indep['random forest'] = erf_result
test_indep['ada boost'] = eab_result

print test_indep.to_latex(index=True, header=True)